# GEOEventFusion — Developer Sandbox

Experimental scratch notebook for iterating on individual pipeline components  
without running the full pipeline. Use this to:

- Test individual agent logic in isolation
- Inspect raw GDELT API responses
- Prototype new analysis functions
- Debug specific pipeline phases

**This notebook is NOT the canonical entry point.** See `quickstart.ipynb` for production use.

## Setup

In [ ]:
import sys
import json
import logging
from pathlib import Path

# Ensure project root is on path when running from notebooks/
_ROOT = Path().resolve().parent
if str(_ROOT) not in sys.path:
    sys.path.insert(0, str(_ROOT))

logging.basicConfig(level=logging.INFO, format='%(levelname)-8s %(name)s — %(message)s')
print(f'Project root: {_ROOT}')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from config.settings import PipelineConfig

# Minimal test config — no real API calls
config = PipelineConfig(
    query='Houthi Red Sea attacks',
    days_back=30,
    llm_backend='ollama',
    max_records=50,
    test_mode=True,
    log_level='DEBUG',
)
print('Config loaded:', config.query)

## Test GDELT Client

In [ ]:
# Direct GDELT API call — inspect raw response
from geoeventfusion.clients.gdelt_client import GDELTClient

client = GDELTClient(
    max_retries=config.gdelt_max_retries,
    backoff_base=config.gdelt_backoff_base,
    request_timeout=config.gdelt_request_timeout,
)

# Fetch a small article list
# response = client.fetch(
#     query='Houthi Red Sea',
#     mode='ArtList',
#     max_records=10,
#     sort='DateDesc',
#     timespan='7d',
# )
# print(json.dumps(response, indent=2)[:2000])
print('GDELTClient instantiated (uncomment the fetch call to run a live query)')

## Test Spike Detector

In [ ]:
from geoeventfusion.analysis.spike_detector import detect_spikes
from geoeventfusion.models.events import TimelineStep

# Build a synthetic timeline with one clear spike
steps = [
    TimelineStep(date=f'2024-01-{i:02d}', value=2.0)
    for i in range(1, 28)
]
steps[14] = TimelineStep(date='2024-01-15', value=9.5)  # Spike
steps[24] = TimelineStep(date='2024-01-25', value=8.0)  # Second spike

spikes = detect_spikes(steps, z_threshold=1.5)
print(f'Detected {len(spikes)} spikes:')
for s in spikes:
    print(f'  [{s.rank}] {s.date}  Z={s.z_score:.2f}  vol={s.volume}')

## Test Actor Graph

In [ ]:
from geoeventfusion.analysis.actor_graph import build_actor_graph

triples = [
    ('Houthi', 'United States', '2024-01-15'),
    ('Houthi', 'Yemen', '2024-01-15'),
    ('United States', 'United Kingdom', '2024-01-16'),
    ('Iran', 'Houthi', '2024-01-17'),
    ('Houthi', 'United States', '2024-01-18'),
    ('Iran', 'United States', '2024-01-19'),
]

graph = build_actor_graph(triples, hub_top_n=3, broker_ratio_threshold=0.5)
print(f'Nodes: {len(graph.nodes)}  Edges: {len(graph.edges)}')
print()
for node in sorted(graph.nodes, key=lambda n: n.pagerank, reverse=True)[:5]:
    print(f'  {node.name:<25} role={node.role:<12} pagerank={node.pagerank:.4f}')

## Test Query Builder

In [ ]:
from geoeventfusion.analysis.query_builder import QueryBuilder

qb = QueryBuilder(
    base_query='Houthi Red Sea attacks',
    repeat_threshold=3,
    near_window=15,
    near_min_term_length=5,
    tone_negative_threshold=-5.0,
    toneabs_threshold=8.0,
)

# Build query variants
print('Repeat query:    ', qb.build_repeat_query())
print('Tone-neg query:  ', qb.build_tone_negative_query())
print('High-emotion:    ', qb.build_high_emotion_query())

## Test LLM Client

In [ ]:
from geoeventfusion.clients.llm_client import LLMClient

# Instantiate (no call made yet)
llm = LLMClient(
    backend=config.llm_backend,
    anthropic_model=config.anthropic_model,
    ollama_model=config.ollama_model,
    ollama_host=config.ollama_host,
    anthropic_api_key=config.anthropic_api_key,
    max_confidence=config.max_confidence,
)
print(f'LLMClient backend: {llm.backend}')
print(f'Max confidence cap: {llm.max_confidence}')

# Uncomment to make a live test call:
# response = llm.call(
#     system='You are a geopolitical analyst.',
#     prompt='In one sentence, what is the Houthi movement?',
#     max_tokens=100,
# )
# print('LLM response:', response)

## Inspect Fixture Data

In [ ]:
# Load and inspect the test fixtures
fixtures_dir = _ROOT / 'tests' / 'fixtures'

with open(fixtures_dir / 'sample_artlist.json', encoding='utf-8') as f:
    artlist = json.load(f)

articles = artlist.get('articles', [])
print(f'Fixture articles: {len(articles)}')
for a in articles[:3]:
    print(f'  [{a.get("seendate", "")}] {a.get("title", "")[:70]}')

## Run Full Pipeline (optional)

In [ ]:
# Uncomment to run the full pipeline with test fixtures (no real API calls)
# from geoeventfusion.pipeline import run_pipeline
# context = run_pipeline(config)
# print(f'Run ID: {context.run_id}')
# print(f'Warnings: {context.warnings}')
print('Uncomment the block above to run the full pipeline in test mode.')